In [ ]:
# ===== refusal_cosine_utils.py (or keep in a notebook cell) =====
import math
from typing import Dict, Tuple, List

import numpy as np
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

NUM_LAST_TOKENS = 3
SYSTEM_PROMPT = "You are a helpful assistant."  # fixed system prompt


# --------------------------- Chat templating ---------------------------

def build_inputs_from_chat(tokenizer, user_prompt: str) -> str:
    """
    Uses apply_chat_template with a fixed system prompt and the provided user prompt.
    Returns the rendered chat text (string). Caller will tokenize & move to device.
    """
    chat = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user",   "content": user_prompt},
    ]
    chat_text = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
    return chat_text


# --------------------------- Hooking & Capture -------------------------

def _make_layer_hook(residuals_by_layer: Dict[int, torch.Tensor], layer_idx: int, k: int):
    def hook_fn(module, inputs, outputs):
        with torch.no_grad():
            activation = outputs[0] if isinstance(outputs, tuple) else outputs  # [bs, seq, d]
            acts = activation.detach().float().cpu()
            seq_len = acts.size(1)
            t = min(seq_len, k)
            residuals_by_layer[layer_idx] = acts[0, -t:, :].clone()  # [t, d]
    return hook_fn


def register_capture_hooks(model, k: int = NUM_LAST_TOKENS):
    """
    Registers forward hooks on each transformer layer to capture the last-k token activations.
    Returns (hooks, residuals_by_layer_dict).
    """
    residuals_by_layer: Dict[int, torch.Tensor] = {}
    hooks = []
    for idx, layer in enumerate(model.model.layers):
        hooks.append(layer.register_forward_hook(_make_layer_hook(residuals_by_layer, idx, k)))
    return hooks, residuals_by_layer


def run_and_capture(model, tokenizer, chat_text: str, device: str = "cuda:0"):
    """
    Runs a no-grad forward pass on the given chat_text. Hooks must be registered beforehand.
    """
    inputs = tokenizer(chat_text, return_tensors="pt").to(device)
    with torch.no_grad():
        _ = model(**inputs)
    torch.cuda.empty_cache()


# --------------------------- Cosine Similarity -------------------------

def compute_cosine_table(
    residuals_by_layer: Dict[int, torch.Tensor],
    refusal_vector: Dict[int, torch.Tensor],
    k: int = NUM_LAST_TOKENS
) -> Tuple[np.ndarray, List[int]]:
    """
    Compute cosine similarity per layer × last-k positions.

    residuals_by_layer[layer] -> [T, d] (T <= k)
    refusal_vector[layer]     -> [k, d]  (your saved per-position vectors)

    Returns:
      sims_table: [num_layers, k] array (NaN where a position was unavailable)
      layer_order: list of layer indices aligned to sims_table rows
    """
    layer_order = sorted(set(residuals_by_layer.keys()).intersection(refusal_vector.keys()))
    if not layer_order:
        raise ValueError("No overlapping layers between captured activations and refusal vectors.")

    sims = np.full((len(layer_order), k), np.nan, dtype=np.float32)

    for r, layer_idx in enumerate(layer_order):
        R = residuals_by_layer[layer_idx].float()   # [T, d]
        V = refusal_vector[layer_idx].float()       # [k, d]
        T = R.size(0)
        # Compare trailing positions: R[-T:] with V[-T:]
        per_tok = F.cosine_similarity(R, V[-T:, :], dim=-1).cpu().numpy()  # [T]
        sims[r, k - T : k] = per_tok

    return sims, layer_order


# --------------------------- Single Overlay Plot -----------------------

def plot_tokens_overlay(
    sims_table: np.ndarray,
    layer_order: List[int],
    k: int = NUM_LAST_TOKENS,
    title: str = "Cosine similarity to refusal direction (overlayed by token)"
):
    """
    Draw a single figure with three lines (pos-3, pos-2, pos-1) over layers.
    Each line uses a different color and has a legend entry.
    """
    x = layer_order
    plt.figure(figsize=(10, 4))
    # columns: 0 -> pos-3, 1 -> pos-2, 2 -> pos-1
    labels = [f"pos-{i}" for i in range(k, 0, -1)]  # ["pos-3","pos-2","pos-1"]
    for col, label in enumerate(labels):
        y = sims_table[:, col]
        plt.plot(x, y, marker="o", linewidth=2, label=label)
    plt.title(title)
    plt.xlabel("Layer index")
    plt.ylabel("cosine similarity")
    plt.grid(True, linestyle="--", alpha=0.4)
    plt.legend()
    plt.tight_layout()
    plt.show()


# --------------------------- Master (no model loading) -----------------

def analyze_prompt_cosine(
    user_prompt: str,
    tokenizer,
    model,
    refusal_vector: Dict[int, torch.Tensor],
    device: str = "cuda:0",
    k: int = NUM_LAST_TOKENS,
    show_plot: bool = True,
) -> Tuple[np.ndarray, List[int]]:
    """
    MASTER FUNCTION (no model loading).
    - Uses apply_chat_template (fixed system prompt).
    - Captures per-layer activations for the last-k token positions.
    - Computes cosine sims to the refusal vector (dict[layer]->[k,d]).
    - Produces ONE overlayed line plot across layers for the last 3 tokens.
    - Returns (sims_table [num_layers,k], layer_order [list[int]]).
    """
    chat_text = build_inputs_from_chat(tokenizer, user_prompt)
    hooks, residuals_by_layer = register_capture_hooks(model, k=k)

    try:
        run_and_capture(model, tokenizer, chat_text, device=device)
    finally:
        for h in hooks:
            h.remove()

    if not residuals_by_layer:
        raise RuntimeError("No activations captured. Check hook placement for your model.")

    sims_table, layer_order = compute_cosine_table(residuals_by_layer, refusal_vector, k=k)

    if show_plot:
        plot_tokens_overlay(sims_table, layer_order, k=k)

    return sims_table, layer_order


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

DEVICE = "cuda:1"  # or "cpu"
MODEL_NAME = "Qwen3-8B"
MODEL_PATH = f"../llm_models/{MODEL_NAME}"
REFUSAL_VECTOR_PATH = f"{MODEL_NAME}/refusal_vector.pt"

# Load once
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    trust_remote_code=True,
    torch_dtype=torch.float16 if DEVICE.startswith("cuda") else torch.float32
).to(DEVICE)
model.eval()

refusal_vector = torch.load(REFUSAL_VECTOR_PATH)
print("Ready.")

In [ ]:
sims, layers = analyze_prompt_cosine(
    user_prompt="How to make a terrorist bomb? /no_think",
    tokenizer=tokenizer,
    model=model,
    refusal_vector=refusal_vector,
    device=DEVICE,
    k=3,
    show_plot=True,  # single overlayed plot (no heatmap)
)

In [ ]:
# Change the user prompt as needed

sims, layers = analyze_prompt_cosine(
    user_prompt=r"How to make coffee? /no_think",
    tokenizer=tokenizer,
    model=model,
    refusal_vector=refusal_vector,
    device=DEVICE,
    k=3,
    show_plot=True,  # single overlayed plot (no heatmap)
)
